# COMS 4995_002 Deep Learning Assignment 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import tensorflow as tf
import glob
import sys
# you shouldn't need to make any more imports

In [ ]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [ ]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

## Create the model

In [ ]:
def Basic_CNN1(features, labels, mode):
    """
    Defines BasicCNN1 architecture.
    This model roughly parallels the simple architecture in the Tensforflow 
    examples code, provided with Tensorflow Python library:
    ...\tensorflow-r1.3\tensorflow\examples\tutorials\mnist
    
    But with the setup modified for CIFAR10, 
    and various other modifications for performance.
    """
    
    #[batch_size, width, height, channels]
    # CIFAR10 are 32 x 32 x 3
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

    #conv1 convolution layer
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=8,
                             kernel_size=[3,3],
                             padding="same",
                             activation=tf.nn.relu,
                             name='conv1')

    # Pooling Layer #1
    # Max pooling, 2x2 filter, stride 2
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name='pool1')

    #conv2 convolution layer
    NF2 = 16
    conv2 = tf.layers.conv2d(inputs=pool1,
                             filters=NF2,
                             kernel_size=[3,3],
                             padding="same",
                             activation=tf.nn.relu,
                             name='conv2')

    # Pooling Layer #2
    #Max pooling, 2x2 filter, stride 2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name='pool2')
    
    #Now do some dense layers with dropout.
    #In order to get to right format for dense, must first flatten:
    #K1 = tf.shape(pool2)[1]
    #K2 = tf.shape(pool2)[2]
    #pool2_flat = tf.reshape(pool2, [-1, K1*K2*NF2, 1])
    pool2_flat = tf.contrib.layers.flatten(pool2)#Had issue with above
    
    # Dense Layer
    dense1 = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu, name='dense1')
    
    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN, name='dropout1')
    
    
    
    
    # Logits layer
    logits = tf.layers.dense(inputs=dropout, units=10, name='logits')

    # Generate predictions (for PREDICT and EVAL mode)
    predictions = {"classes": tf.argmax(input=logits, axis=1), 
                   "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
                   }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)


    #Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels,
                                           logits=logits)


    # Training (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels,
                                                       predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)


## Load the data, using utility functions from HW1.

In [ ]:
# Load the data, using utility functions from HW1
data_root_path = 'cifar10-hw1/'
X, y = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

# Hold out 10% of the data to use as a validation set:
train_inds = np.random.choice(X.shape[1],int(X.shape[1]*.90),replace=False)
validation_inds = np.setdiff1d(np.arange(X.shape[1]),train_inds)
X_train = np.moveaxis(X[:,train_inds].reshape(32,32,3,train_inds.size),-1,0).astype(np.float32)
y_train = y[train_inds].astype(np.int32)
X_validation = np.moveaxis(X[:,validation_inds].reshape(32,32,3,validation_inds.size),-1,0).astype(np.float32)
y_validation = y[validation_inds].astype(np.int32)


## Train, test

In [ ]:
MAX_STEPS = 10

# Create the Estimator (using our own model)
cifar10_classifier = tf.estimator.Estimator(model_fn=Basic_CNN1,
                                            model_dir="/tmp/cifar10_CNN")


# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {} # {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log,
                                          every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X_train},
                                                    y=y_train,
                                                    batch_size=100,
                                                    num_epochs=None,
                                                    shuffle=False)
cifar10_classifier.train(input_fn=train_input_fn,
                         steps=MAX_STEPS,
                         hooks=[logging_hook])


#Test on validation set:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X_validation},
                                                   y=y_validation,
                                                   num_epochs=1,
                                                   shuffle=False)
eval_results = cifar10_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
print("Done.")

#    #Make predictions on Test set, and save them out:
#    X_test...
#    #SAVE
#    print('saving')
#    y_predicted3 = aaaaa#cifar10_classifier.evaluate(input_fn=eval_input_fn)
#    save_predictions('ans3-uni',y_predicted3)
#    
#    #CHECK LOADED
#    # test if your numpy file has been saved correctly
#    loaded_y3 = np.load('ans3-uni.npy')
#    print(loaded_y3.shape)
#    loaded_y3[:10]